# Requerimientos

In [1]:
print("Hello There")

Hello There


In [1]:
import os
from tkinter import Tk, filedialog
from numpy import repeat
import cv2
from win32con import (SW_SHOW, SW_RESTORE)
import win32gui
import win32ui

Si hay problemas al ejecutarse desde terminal, utilizar el comando:
pip install pywin32 --upgrade

In [2]:
inicio=win32ui.MessageBox("Para reetiquetar las imagenes de acuerdo a la posición del item, utiliza las teclas: \n\n W: Frontal \n S: Trasera \n A: Izquierda \n D: Derecha", "Instrucciones Generales",1)
if inicio==2:
    exit()

consideraciones = win32ui.MessageBox("La carpeta debe contener archivos JPG ya etiquetados\n\nEl nombre de la carpeta no debe contener ( _ )\n\nUna ventana emergente mostrará la vista elegida, si desea cambiarla presione la opción Cancelar y escoja nuevamente entre W,A,S o D\n\nUna vez inciado el programa podrá salir hasta que se hayan reetiquetado todas las imagenes de la carpeta", "Consideraciones",1)
if consideraciones==2:
    exit()

La carpeta que se seleccionará debe de incluir las imagenes que se reetiquetarán ya que es muy probable que el modelo se haya equivocado en estos casos. Una consideración importante es que el nombre de la carpeta no debe incluir "_" pues el programa toma en consideración todo el path

Abriendo carpeta que contiene las imagenes desde ventana Windows

In [3]:
root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
carpeta = filedialog.askdirectory() # Returns opened path as str


In [4]:
# get base name of carpeta
base_name_carpeta = os.path.basename(carpeta)

#base_name_carpeta contains "_"
if "_" in base_name_carpeta:
    d=win32ui.MessageBox("El nombre de la carpeta contiene ( _ ) desea cambiarlo? ", "Error", 1) #1 Aceptar 2 Cancelar
    if d==1:
        #rename carpeta path
        os.rename(carpeta, carpeta.replace("_", ""))
        carpeta=carpeta.replace("_", "")
        #base_name_carpeta = base_name_carpeta.replace("_", "")
    elif d==2:
        exit()
    


In [5]:
os.chdir(carpeta) # Changes directory to opened path.
files = os.listdir()

# if the folder is empty, the program will exit
if len(files) == 0:
    win32ui.MessageBox("La carpeta esta vacia", "Error",0)
    exit()

if not all(file.endswith('.jpg') for file in files):
    win32ui.MessageBox("La carpeta solo debe de contener archivos JPG", "Error",0)
    exit()    
    

Seleccionando solo las imagenes con medida 515 para mostrarse en pantalla

In [6]:
# get path of all files
paths = [os.path.join(carpeta, file) for file in files]

#Selecting all paths with 515x515H in them
principales = [path for path in paths if "515Wx515" in path]

# Open cv2

Para que el uso del programa sea mas fluido en el archivo Ejecutable.py se agregan las dos ultimas funciones las cuales colocan sobre el resto de ventanas la imagen a reetiquetar y al mismo tiempo se activa dicha ventana.

In [7]:
#Functions --------------------------------------------------------------------------------------------------
# this function is used to create a subset from a string
def get_subset(string, start, end):
    return string[start:end]    

# this function detects the position of a  character in a string
def detect_char(string, char):
    return [i for i, letter in enumerate(string) if letter == char]  

# With these functions we can activate the window and bring it to the front.
def get_windows_placement(window_id):
    return win32gui.GetWindowPlacement(window_id)[1]

def set_active_window(window_id):
    if get_windows_placement(window_id) == 2:
        win32gui.ShowWindow(window_id, SW_RESTORE)
    else:
        win32gui.ShowWindow(window_id, SW_SHOW)
    win32gui.SetForegroundWindow(window_id)
    win32gui.SetActiveWindow(window_id)


* Mostrando las imagenes y utilizando las teclas (w,a,s,d) para etiquetarlas
* Se muestra la imagen y posteriormente la etiqueta que se le asignó mediante el teclado para que el usuario corrobore su selección
* Incluye un bucle que se rompe hasta que se selecciona la opción aceptar (decision final)
* Se renombran con el formato "-new" para evitar repeticiones con los archivos ya existentes

In [8]:
def seleccion(imagen): 
    img = cv2.imread(imagen)
    cv2.imshow('Imagen',img) #
    cv2.setWindowProperty('Imagen', cv2.WND_PROP_TOPMOST, 1)
    window_id = win32gui.GetActiveWindow()#
    set_active_window(window_id)#
    #detecting which key is pressed 
    tecla=cv2.waitKey(0) # waits until a key is pressed
    
    # Pressing only keys predefined
    while tecla != ord('a') and tecla != ord('d') and tecla != ord('w') and tecla != ord('s'):
        win32ui.MessageBox("Recuerda que para etiquetar las fotos debes de presionar las teclas \t W A S D", "Cuidado", 0)
        img = cv2.imread(imagen)
        cv2.imshow('Imagen',img) #
        cv2.setWindowProperty('Imagen', cv2.WND_PROP_TOPMOST, 1)
        window_id = win32gui.GetActiveWindow()#
        set_active_window(window_id)#
        tecla=cv2.waitKey(0) 
        cv2.destroyAllWindows() 

    # Conditional statement to detect which key is pressed
    if tecla == ord('a'):
        new_label="_Izquierda-new_"
    elif tecla == ord('d'):
        new_label="_Derecha-new_" 
    elif tecla == ord('w'):
        new_label="_Frontal-new_"
    elif tecla == ord('s'):
        new_label="_Trasera-new_"
    
    return new_label
      

+ Si se denominan dos imagenes como "frontales" por ejemplo, la primera conserva su nombre y la segunda cambia su no_vista 
+ Solo se aceptan dos imagenes del mismo tipo, (p ejem 2 izquierdas)


In [9]:
for i in range(len(principales)):
    imagen = principales[i]
    
    new_label=seleccion(imagen)
    
    #Now, if the user wants to change his/her selection, he/ she can select the option "Cancel"
    continuar = win32ui.MessageBox(new_label, "Seleccion", 1) #?

    while continuar == 2:
        new_label=seleccion(imagen)
        continuar = win32ui.MessageBox(new_label, "Seleccion", 1)

    cv2.destroyAllWindows() # destroys the window showing image  
      
    # Elements of the path 
    tipo=get_subset( imagen, detect_char(imagen, '_')[0], detect_char(imagen, '_')[1]+1) #_Derecha_
    sku =get_subset( imagen, detect_char(imagen, '_')[1]+1, detect_char(imagen, '_')[2]) #1700490094
    no_vista=get_subset(imagen, detect_char(imagen, '_')[2], detect_char(imagen, '_')[2]+3) #_1.

    item = [path for path in paths if sku in path] 
    vista=[item for item in item if tipo in item] 

    # Selecting all paths with sku in them and with determinated vista
    item_vista=[vista for vista in vista if no_vista in vista] 
    
    # if imagen2.replace(tipo, new_label) does not exist, rename imagen2 to imagen2.replace(tipo, new_label)
    for j in range(len(item_vista)) :
        imagen2 = item_vista[j]
        if not os.path.exists(imagen2.replace(tipo, new_label)):
            os.rename(imagen2, imagen2.replace(tipo, new_label))
        # Here we are checking if the kinds file has variants.
        else:
            if not os.path.exists(imagen2.replace(no_vista, "_1.")):
                os.rename(imagen2, imagen2.replace(no_vista, "_1."))
            else:
                if not os.path.exists(imagen2.replace(no_vista, "_2.")):
                    os.rename(imagen2, imagen2.replace(no_vista, "_2.")) 
                else:
                    print("Ya existe")

Se revisa que todos los nombres se actualizaron, si no es así, es porque se asigaron mas formatos de los que son permitidos (por ejemplo se seleccionaron 3 imagenes como frontales, sin embargo solo se aceptan 2). Si es el caso, se mueve hacia una direccion que vivirá al mismo nivel que la carpeta padre.

In [10]:
files=os.listdir()
for i in range(len(files)):
    if '-' not in files[i]:
        base_name=carpeta[:-len(os.path.basename(carpeta))]
        if not os.path.exists(base_name+"Repeticiones"):
            os.mkdir(base_name+"Repeticiones")
        os.rename(files[i], base_name+"Repeticiones/"+files[i])
files= os.listdir()
for i in range(len(files)):
    os.rename(files[i],files[i].replace('-new', ''))


Finalmente se eliminan todas las varibales que fueron creadas, se libera memoria y evita complicaciones en posteriores usos 

In [11]:
del  carpeta, files, paths, principales, imagen, new_label